In [ ]:
!pip install mlflow optuna imbalanced-learn

In [ ]:
import os
import mlflow

# Replace with your actual username and token
os.environ["MLFLOW_TRACKING_USERNAME"] = "sreyo2004"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "54b0fc2a87e30ce5bfc3a8ed37bac2ef6d65c1ad"

mlflow.set_tracking_uri("https://dagshub.com/sreyo2004/mlflow-test.mlflow")

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Himanshu-1703/reddit-sentiment-analysis/refs/heads/main/data/reddit.csv')
df.head()

clean_comment  category
0   family mormon have never tried explain them t...         1
1  buddhism has very much lot compatible with chr...         1
2  seriously don say thing first all they won get...        -1
3  what you have learned yours and only yours wha...         0
4  for your own benefit you may want read living ...         1

In [ ]:
# Drop missing and duplicate values
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df = df[~(df['clean_comment'].str.strip() == '')]

# Import necessary libraries
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')

# Define the preprocessing function
def preprocess_comment(comment):
    # Convert to lowercase
    comment = comment.lower()

    # Remove trailing and leading whitespaces
    comment = comment.strip()

    # Remove newline characters
    comment = re.sub(r'\n', ' ', comment)

    # Remove non-alphanumeric characters, except punctuation
    comment = re.sub(r"[^A-Za-z0-9\s!?.]", '', comment)

    # Remove stopwords but retain important ones for sentiment analysis
    stop_words = set(stopwords.words('english')) - {'not', 'but', 'however', 'no', 'yet'}
    comment = ' '.join([word for word in comment.split() if word not in stop_words])

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    comment = ' '.join([lemmatizer.lemmatize(word) for word in comment.split()])

    return comment

# Apply the preprocessing function to the `clean_comment` column
df['clean_comment'] = df['clean_comment'].apply(preprocess_comment)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna

In [ ]:
mlflow.set_experiment("Exp 5 - ML Algos with HP Tuning")

<Experiment: artifact_location='mlflow-artifacts:/57a11b57c17f4fbf87cf3fa6ca38890d', creation_time=1749279996336, experiment_id='9', last_update_time=1749279996336, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={}>

In [ ]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for Random Forest

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)




# Step 6: Optuna objective function for Random Forest
def objective_rf(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)  # Number of trees in the forest
    max_depth = trial.suggest_int('max_depth', 3, 20)  # Maximum depth of the tree
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)  # Minimum samples required to split a node
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)  # Minimum samples required at a leaf node

    # RandomForestClassifier setup
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth,
                                   min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf,
                                   random_state=42)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for Random Forest, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_rf, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = RandomForestClassifier(n_estimators=best_params['n_estimators'],
                                        max_depth=best_params['max_depth'],
                                        min_samples_split=best_params['min_samples_split'],
                                        min_samples_leaf=best_params['min_samples_leaf'],
                                        random_state=42)

    # Log the best model with MLflow, passing the algo_name as "RandomForest"
    log_mlflow("RandomForest", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for Random Forest
run_optuna_experiment()


[I 2025-06-16 14:49:17,928] A new study created in memory with name: no-name-eb6c3d36-f3ec-429e-ac62-53d0b4fcc9e5
[I 2025-06-16 14:49:29,000] Trial 0 finished with value: 0.6816020289548769 and parameters: {'n_estimators': 247, 'max_depth': 13, 'min_samples_split': 10, 'min_samples_leaf': 12}. Best is trial 0 with value: 0.6816020289548769.
[I 2025-06-16 14:49:31,775] Trial 1 finished with value: 0.6379583641551305 and parameters: {'n_estimators': 213, 'max_depth': 5, 'min_samples_split': 11, 'min_samples_leaf': 13}. Best is trial 0 with value: 0.6816020289548769.
[I 2025-06-16 14:49:36,859] Trial 2 finished with value: 0.6876254887456409 and parameters: {'n_estimators': 135, 'max_depth': 17, 'min_samples_split': 15, 'min_samples_leaf': 17}. Best is trial 2 with value: 0.6876254887456409.
[I 2025-06-16 14:49:45,994] Trial 3 finished with value: 0.6880481876783261 and parameters: {'n_estimators': 263, 'max_depth': 16, 'min_samples_split': 10, 'min_samples_leaf': 9}. Best is trial 3 with

🏃 View run RandomForest_SMOTE_TFIDF_Trigrams at: https://dagshub.com/sreyo2004/mlflow-test.mlflow/#/experiments/9/runs/0061238cfe1a4ef2b9875da8f29973eb
🧪 View experiment at: https://dagshub.com/sreyo2004/mlflow-test.mlflow/#/experiments/9
